In [61]:
from rasa.core.tracker_store import (
    TrackerStore,
    InMemoryTrackerStore,
    RedisTrackerStore)
from rasa.core.events import (
    SlotSet,
    ActionExecuted,
    Restarted,
    UserUttered,
    SessionStarted,
    BotUttered,
    Event,
)
from rasa.core.trackers import DialogueStateTracker
from rasa.core.domain import Domain

def get_default_domain():
    content = """
    actions:
      - utter_hello

    intents:
      - greet
      - bye
      - affirm
      - deny
    entities:
      - object_type
      - mention
      - attribute
      - sents

    slots:
      attribute:
        type: unfeaturized
      mention:
        type: unfeaturized
      object:
        type: unfeaturized
      object_type:
        type: unfeaturized
      parameters:
        type: unfeaturized
      sents:
        type: unfeaturized
    """
    return Domain.from_yaml(content)

default_domain=get_default_domain()
tracker_store = RedisTrackerStore(
        domain=default_domain,
        host="localhost",
        port=6379,
        db=0,
        record_exp=3000,
    )

In [40]:
events = [
    UserUttered("Hola", {"name": "greet"}),
    BotUttered("Hi"),
    SessionStarted(),
    UserUttered("Ciao", {"name": "greet"}),
]
sender_id = "test_sql_tracker_store_with_session_events"
tracker = DialogueStateTracker.from_events(sender_id, events)
tracker_store.save(tracker)

In [57]:
tracker = tracker_store.retrieve(sender_id)
tracker.events

deque([<rasa.core.events.UserUttered at 0x11405cfd0>,
       BotUttered('Hi', {}, {}, 1582039291.9532218),
       <rasa.core.events.Restarted at 0x11405cef0>])

In [58]:
tracker.current_slot_values()

{'attribute': None,
 'mention': None,
 'object': None,
 'object_type': None,
 'parameters': None,
 'sents': None}

In [15]:
tracker_store.serialise_tracker(tracker)

'{"events": [{"event": "user", "timestamp": 1582037250.804071, "text": "Hola", "parse_data": {"intent": {"name": "greet"}, "entities": [], "text": "Hola", "message_id": null, "metadata": {}}, "input_channel": null, "message_id": null, "metadata": {}}, {"event": "bot", "timestamp": 1582037250.80408, "text": "Hi", "data": {}, "metadata": {}}, {"event": "session_started", "timestamp": 1582037250.8040829}, {"event": "user", "timestamp": 1582037250.804086, "text": "Ciao", "parse_data": {"intent": {"name": "greet"}, "entities": [], "text": "Ciao", "message_id": null, "metadata": {}}, "input_channel": null, "message_id": null, "metadata": {}}], "name": "test_sql_tracker_store_with_session_events"}'

In [16]:
dialogue = tracker.as_dialogue()
dialogue.as_dict()

{'events': [{'event': 'user',
   'timestamp': 1582037250.804071,
   'text': 'Hola',
   'parse_data': {'intent': {'name': 'greet'},
    'entities': [],
    'text': 'Hola',
    'message_id': None,
    'metadata': {}},
   'input_channel': None,
   'message_id': None,
   'metadata': {}},
  {'event': 'bot',
   'timestamp': 1582037250.80408,
   'text': 'Hi',
   'data': {},
   'metadata': {}},
  {'event': 'session_started', 'timestamp': 1582037250.8040829},
  {'event': 'user',
   'timestamp': 1582037250.804086,
   'text': 'Ciao',
   'parse_data': {'intent': {'name': 'greet'},
    'entities': [],
    'text': 'Ciao',
    'message_id': None,
    'metadata': {}},
   'input_channel': None,
   'message_id': None,
   'metadata': {}}],
 'name': 'test_sql_tracker_store_with_session_events'}

In [59]:
# preqs: 
# $ saai talk 'hello' samlet
# $ saai talk '/dump_info{"object":"rr"}' samlet
tracker = tracker_store.retrieve('samlet')
# tracker.events
dialogue = tracker.as_dialogue()
dialogue.as_dict()

{'events': [{'event': 'action',
   'timestamp': 1582039814.724075,
   'name': 'action_listen',
   'policy': None,
   'confidence': None},
  {'event': 'user',
   'timestamp': 1582039823.995178,
   'text': '/dump_info{"object":"rr"}',
   'parse_data': {'text': '/dump_info{"object":"rr"}',
    'intent': {'name': 'dump_info', 'confidence': 1.0},
    'intent_ranking': [{'name': 'dump_info', 'confidence': 1.0}],
    'entities': [{'entity': 'object', 'start': 10, 'end': 25, 'value': 'rr'}]},
   'input_channel': 'rest',
   'message_id': '2f71e730f2374a439fa77f82ed76eab8',
   'metadata': {}},
  {'event': 'slot',
   'timestamp': 1582039823.995191,
   'name': 'object',
   'value': 'rr'},
  {'event': 'action',
   'timestamp': 1582039824.004002,
   'name': 'action_dump_info',
   'policy': 'policy_2_MappingPolicy',
   'confidence': 1},
  {'event': 'action',
   'timestamp': 1582039824.010093,
   'name': 'action_listen',
   'policy': 'policy_2_MappingPolicy',
   'confidence': 1},
  {'event': 'user',
 

In [60]:
tracker.current_slot_values()

{'attribute': None,
 'mention': None,
 'object': 'rr',
 'object_type': None,
 'parameters': None,
 'sents': None}

In [45]:
slot_key = "object"
slot_val = "Easter Island"
# tracker = tracker_store.get_or_create_tracker('samlet')
ev = SlotSet(slot_key, slot_val)
tracker.update(ev)
tracker_store.save(tracker)

In [55]:
again = tracker_store.get_or_create_tracker('samlet')
again.get_slot(slot_key)

'rr'

In [52]:
ev = Restarted()
tracker.update(ev)
tracker_store.save(tracker)

In [54]:
tracker=tracker_store.create_tracker('samlet')
tracker_store.save(tracker)

* refs
- ⊕ [Responses](https://rasa.com/docs/rasa/core/responses/)

```json
{
  "tracker": {
    "latest_message": {
      "text": "/greet",
      "intent_ranking": [
        {
          "confidence": 1.0,
          "name": "greet"
        }
      ],
      "intent": {
        "confidence": 1.0,
        "name": "greet"
      },
      "entities": []
    },
    "sender_id": "22ae96a6-85cd-11e8-b1c3-f40f241f6547",
    "paused": false,
    "latest_event_time": 1531397673.293572,
    "slots": {
      "name": null
    },
    "events": [
      {
        "timestamp": 1531397673.291998,
        "event": "action",
        "name": "action_listen"
      },
      {
        "timestamp": 1531397673.293572,
        "parse_data": {
          "text": "/greet",
          "intent_ranking": [
            {
              "confidence": 1.0,
              "name": "greet"
            }
          ],
          "intent": {
            "confidence": 1.0,
            "name": "greet"
          },
          "entities": []
        },
        "event": "user",
        "text": "/greet"
      }
    ]
  },
  "arguments": {},
  "template": "utter_greet",
  "channel": {
    "name": "collector"
  }
}
```